In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow.keras.utils as ku  
from wordcloud import WordCloud 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers  
from sklearn.model_selection import train_test_split 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf 
from tensorflow.keras.regularizers import l2

In [ ]:
file_path = '/kaggle/input/robert-frost-poems-dataset-cleaned/cleaned_data.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)  

new_header = ['title']  
data.columns=new_header
data

In [ ]:
data['title'] = data['title'].apply(lambda x: x.replace(u'\xa0',u' '))
data['title'] = data['title'].apply(lambda x: x.replace('\u200a',' ')) 


In [ ]:
tokenizer = Tokenizer(oov_token='<oov>') 
tokenizer.fit_on_texts(data['title'])
total_words = len(tokenizer.word_index) + 1

In [ ]:
sentences=data['title'].tolist()
input_sequences = []
for sentence in sentences:
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]  
        input_sequences.append(n_gram_sequence)


max_sequence_len = max(len(seq) for seq in input_sequences)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

y = tf.keras.utils.to_categorical(y, num_classes=total_words) 

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])  

history = model.fit(X_train, y_train, epochs=12, verbose=1, batch_size=16, validation_split=0.2)
print(model)

In [ ]:
seed_text = "woods are lovely"
next_words = 25
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    predicted=model.predict(token_list) 
    predicted=np.argmax(predicted,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

In [ ]:
def accuracy(X, y, model):  
    total_samples=X.shape[0]
    y_pred=model.predict(X)
    correct_samples=0
    for i in range(X.shape[0]): 
        y1=np.argmax(y[i])
        y2=np.argmax(y_pred[i])
        if (y1==y2): 
            correct_samples+=1
            
    return correct_samples/total_samples 


acc1=accuracy(X_test, y_test, model) 
print(acc1)

In [ ]:
def calculate_perplexity(model, X_test, y_test):
    predictions = model.predict(X_test)  
    
    cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_test, predictions)
    perplexity = np.exp(np.mean(cross_entropy_loss))
    return perplexity

perplexity = calculate_perplexity(model, X_test, y_test)
print(perplexity)